In [4]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
#configuración en google colab
#instalar java y spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz
!tar xf spark-3.0.2-bin-hadoop3.2.tgz
!pip install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"
import findspark
findspark.init()

In [6]:
# verificar que tengan instalado la librería 'pyspark'
!pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 20.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=36d6ce26b372f52907f3173c148ae73186d3ea0e452a560a1ed09a1f9c2de417
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [7]:
from pyspark.sql import SparkSession

#forma 1 de crear la sesión y el contexto Spark:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

#forma 2 de crear la sesión y el contexto Spark:
#sc = SparkContext.getOrCreate()
#spark=SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
#myrdd = sc.wholeTextFiles('../datasets/papers_sample_pdf/*.txt')
#df = myrdd.toDF(schema=['filename','content'])
#df.show(5)

In [8]:
Path_in = '/content/gdrive/MyDrive/Colab_Notebooks/st1800_20211/datasets/papers_sample_pdf/'


In [9]:
textFiles = spark.sparkContext.wholeTextFiles(Path_in+"*.txt")    
spark_df_folder = textFiles.toDF(schema=['path','text'])
df=spark_df_folder.select('text')
df.show(8)

+--------------------+
|                text|
+--------------------+
|Smooth Rényi Ent...|
|Analyzing Design ...|
|1

Colour Image S...|
|1

Unequal Error ...|
|On the hitting ti...|
|arXiv:0811.1254v1...|
|Generating Random...|
|1

Variations on ...|
+--------------------+



In [10]:
df.printSchema()
df.show(6)

root
 |-- text: string (nullable = true)

+--------------------+
|                text|
+--------------------+
|Smooth Rényi Ent...|
|Analyzing Design ...|
|1

Colour Image S...|
|1

Unequal Error ...|
|On the hitting ti...|
|arXiv:0811.1254v1...|
+--------------------+
only showing top 6 rows



In [12]:
# Tokenization
from pyspark.ml.feature import Tokenizer
import re

tokenization=Tokenizer(inputCol='text',outputCol='tokens')
tokenized_df=tokenization.transform(df)
#tokenized_df = tokenized_df.withColumn('tokens', re.subn('[^A-Za-z0-9]+', '','tokens'))
tokenized_df.printSchema()
tokenized_df.show(8)


root
 |-- text: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------+--------------------+
|                text|              tokens|
+--------------------+--------------------+
|Smooth Rényi Ent...|[smooth, rényi, ...|
|Analyzing Design ...|[analyzing, desig...|
|1

Colour Image S...|[1, , colour, ima...|
|1

Unequal Error ...|[1, , unequal, er...|
|On the hitting ti...|[on, the, hitting...|
|arXiv:0811.1254v1...|[arxiv:0811.1254v...|
|Generating Random...|[generating, rand...|
|1

Variations on ...|[1, , variations,...|
+--------------------+--------------------+



In [15]:
##Tokenización en bi-grams 
from pyspark.ml.feature import NGram

ngram_df = NGram(n=2, inputCol="tokens", outputCol="bigrams").transform(tokenized_df)
ngram_df.select(['bigrams']).show()

+--------------------+
|             bigrams|
+--------------------+
|[smooth rényi, r...|
|[analyzing design...|
|[1 ,  colour, col...|
|[1 ,  unequal, un...|
|[on the, the hitt...|
|[arxiv:0811.1254v...|
|[generating rando...|
|[1 ,  variations,...|
+--------------------+



In [16]:
## Tokenización en tri-grams

##Tokenización en bi-grams 
from pyspark.ml.feature import NGram

ngram_df = NGram(n=3, inputCol="tokens", outputCol="trigrams").transform(tokenized_df)
ngram_df.select(['trigrams']).show()

+--------------------+
|            trigrams|
+--------------------+
|[smooth rényi en...|
|[analyzing design...|
|[1  colour,  colo...|
|[1  unequal,  une...|
|[on the hitting, ...|
|[arxiv:0811.1254v...|
|[generating rando...|
|[1  variations,  ...|
+--------------------+



In [17]:
# stopwords removal 
from pyspark.ml.feature import StopWordsRemover
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
refined_df=stopword_removal.transform(tokenized_df)
refined_df.select(['refined_tokens']).show(8,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
refined_df.columns

['text', 'tokens', 'refined_tokens']

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [22]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_count_df = refined_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [23]:
refined_count_df.orderBy(rand()).show(10)

+--------------------+--------------------+--------------------+-----------+
|                text|              tokens|      refined_tokens|token_count|
+--------------------+--------------------+--------------------+-----------+
|Smooth Rényi Ent...|[smooth, rényi, ...|[smooth, rényi, ...|       3122|
|Analyzing Design ...|[analyzing, desig...|[analyzing, desig...|       2505|
|1

Colour Image S...|[1, , colour, ima...|[1, , colour, ima...|       5158|
|On the hitting ti...|[on, the, hitting...|[hitting, times, ...|       7404|
|arXiv:0811.1254v1...|[arxiv:0811.1254v...|[arxiv:0811.1254v...|      10952|
|1

Variations on ...|[1, , variations,...|[1, , variations,...|       8392|
|Generating Random...|[generating, rand...|[generating, rand...|      13224|
|1

Unequal Error ...|[1, , unequal, er...|[1, , unequal, er...|      20398|
+--------------------+--------------------+--------------------+-----------+



In [ ]:
# Count Vectorizer
from pyspark.ml.feature import CountVectorizer
from matplotlib import pyplot as plt

count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')
cv_df=count_vec.fit(refined_df).transform(refined_df)
cv_df.select(['refined_tokens','features']).show(4,False)

bow = count_vec.fit(refined_df).vocabulary
print(bow)



+--------------------+
|      refined_tokens|
+--------------------+
|[smooth, rényi, ...|
|[analyzing, desig...|
|[1, , colour, ima...|
|[1, , unequal, er...|
|[hitting, times, ...|
|[arxiv:0811.1254v...|
|[generating, rand...|
|[1, , variations,...|
+--------------------+



In [25]:
cv_df.select(['features']).show(4,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
# TF with HashingTF
from pyspark.ml.feature import HashingTF
# podria utilizar numFeatures como el tamaño del Bag of Words:
l = len(bow)
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features',numFeatures=16)
#hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features',numFeatures=11)
# compare la salida e interprete con y sin numFeatures:
#hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

hashing_df=hashing_vec.transform(refined_df)
hashing_df.show(4)



+--------------------+--------------------+--------------------+--------------------+
|                text|              tokens|      refined_tokens|         tf_features|
+--------------------+--------------------+--------------------+--------------------+
|Smooth Rényi Ent...|[smooth, rényi, ...|[smooth, rényi, ...|(16,[0,1,2,3,4,5,...|
|Analyzing Design ...|[analyzing, desig...|[analyzing, desig...|(16,[0,1,2,3,4,5,...|
|1

Colour Image S...|[1, , colour, ima...|[1, , colour, ima...|(16,[0,1,2,3,4,5,...|
|1

Unequal Error ...|[1, , unequal, er...|[1, , unequal, er...|(16,[0,1,2,3,4,5,...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 4 rows



In [28]:
from pyspark.ml.feature import IDF
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)
tf_idf_df.select('tf_idf_features').show(4,False)

+--------------------------------------------------------------------------------------------------------------+
|tf_idf_features                                                                                               |
+--------------------------------------------------------------------------------------------------------------+
|(16,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])|
|(16,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])|
|(16,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])|
|(16,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])|
+--------------------------------------------------------------------------------------------------------------+
only showing top 4 rows



In [30]:
from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol='refined_tokens', outputCol="word2vec")
model = word2Vec.fit(refined_df)

result = model.transform(refined_df.select(['refined_tokens']))
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [smooth, rényi, entropy, ergodic, quantum, information, sources, berry, schoenmakers, , jilles, tjoelker, , arxiv:0704.3504v1, [quant-ph], 26, apr, 2007, , dept., mathematics, computer, science, technical, university, eindhoven, netherlands, berry@win.tue.nl, , j.tjoelker@student.tue.nl, , pim, tuyls, , evgeny, verbitskiy, , information, security, systems, philips, research, eindhoven, netherlands, , digital, signal, processing, philips, research, eindhoven, netherlands, , pim.tuyls@philips.com, , evgeny.verbitskiy@philips.com, , abstract—, investigate, recently, introduced, notion, smooth, rényi, entropy, case, ergodic, information, sources,, thereby, generalizing, previous, work, concentrated, mainly, i.i.d., information, sources., actually, consider, ergodic, quantum, information, sources,, ergodic, classical, information, sources, special, case., prove, average, smooth, rényi, entropy, rate, approach, entropy, rate, stationary,, ergodic, source,, equal, shannon, entropy, r